In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
data =pd.read_csv('XAUUSD.csv')
data

,time,open,high,low,close,tick_volume,MA_daily,MA_weekly,RSI,BB_High,BB_Low,MACD
0,2017-01-31 11:00:00,1198.45,1204.32,1197.53,1204.23,26378,1192.6794,1192.27010,65.694013,1203.664980,1192.537020,2.179186
1,2017-01-31 12:00:00,1204.23,1206.82,1203.68,1204.89,38616,1193.1338,1192.57645,66.653721,1204.512757,1192.861243,2.460562
2,2017-01-31 13:00:00,1204.88,1205.65,1203.27,1204.11,37947,1193.5702,1192.88415,64.362141,1205.260365,1192.700635,2.590749
3,2017-01-31 14:00:00,1204.08,1214.34,1203.94,1212.94,70737,1194.1904,1193.23945,74.887745,1208.366068,1191.385932,3.367612
4,2017-01-31 15:00:00,1212.94,1215.41,1211.77,1212.51,75073,1194.7566,1193.59475,73.745489,1210.654529,1190.619471,3.903585
...,...,...,...,...,...,...,...,...,...,...,...,...
37558,2023-06-15 15:00:00,1953.14,1957.81,1951.95,1956.56,7423,1945.1786,1954.58930,63.967108,1954.626510,1923.696490,-0.000903
37559,2023-06-15 16:00:00,1956.56,1960.20,1956.56,1959.55,5943,1945.1422,1954.65475,66.026741,1957.597705,1922.186295,1.345662
37560,2023-06-15 17:00:00,1959.55,1960.28,1957.71,1958.76,4674,1945.2342,1954.71090,64.970058,1960.234958,1921.203042,2.322307
37561,2023-06-15 18:00:00,1958.78,1959.50,1956.93,1957.52,3697,1945.3326,1954.77385,63.258764,1962.211225,1920.577775,2.962102


In [ ]:

# Step 1: Prepare the Data

# Splitting data into input features and supply/demand zone information
features = ['Opening', 'High', 'Low', 'RSI', 'SMA']
zone_info = ['ZonePresence', 'ZoneLevel']

# Splitting the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Step 2: Data Preprocessing

# Normalize the input features using Min-Max scaler
scaler = MinMaxScaler()
train_data[features] = scaler.fit_transform(train_data[features])
test_data[features] = scaler.transform(test_data[features])

# Convert the zone information into numerical values if necessary
train_data['ZonePresence'] = train_data['ZonePresence'].map({'no zone': 0, 'zone': 1})
test_data['ZonePresence'] = test_data['ZonePresence'].map({'no zone': 0, 'zone': 1})


In [ ]:

# Convert the data into sequences of fixed length
sequence_length = 10

def create_sequences(data):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[features].values[i:i+sequence_length])
        y.append(data[zone_info].values[i+sequence_length])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)
X_test, y_test = create_sequences(test_data)

# Convert NumPy arrays to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()


In [ ]:

# Step 3: Build the LSTM Model

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = len(features)
hidden_size = 64
output_size = len(zone_info)

model = LSTMModel(input_size, hidden_size, output_size)


In [ ]:

# Define hyperparameters
learning_rate = 0.001
batch_size = 32
num_epochs = 50

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

# Create DataLoader objects for training and testing
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    # Print the loss after every epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

# Step 5: Evaluation and Prediction


In [ ]:

# Evaluate the model on the testing set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    predicted_zones = (torch.sigmoid(test_outputs) > 0.5).float()

# Calculate accuracy or other evaluation metrics
accuracy = (predicted_zones == y_test).sum().item() / (y_test.numel())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Make predictions on unseen data
unseen_data = ...  # unseen data to predict on
unseen_data[features] = scaler.transform(unseen_data[features])
X_unseen, _ = create_sequences(unseen_data)
X_unseen = torch.from_numpy(X_unseen).float()

model.eval()
with torch.no_grad():
    unseen_outputs = model(X_unseen)
    unseen_predicted_zones = (torch.sigmoid(unseen_outputs) > 0.5).float()
